In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

#cinfiguration 
num_rows = 50000
filename = " optum_eligibility_practice.csv"

data= {
    'Member_id':[f'M{100000 + i}'for i in range(num_rows)],
    'Subscriber_id':[f'S{100000 + random.randint(0,num_rows//2)}' for _ in range(num_rows)],
    'First_name': [random.choice(['John','Jane','Michael','Sarah','Robert','Emiliy']) for _ in range(num_rows)],
    'Last_name':[random.choice(['Smith','Johnson','Williams','Brown','Jones','Garcia'])for _ in range(num_rows)],
    'DOB':[datetime(1950,1,1)+timedelta(days=random.randint(0,25000)) for _ in  range(num_rows)],
    'Gender': [random.choice(['M','F','U']) for _ in range(num_rows)],
    'Plan_code':[random.choice(['PPO_001', 'HMO_CORE', 'HDHP_SILVER', 'MEDICARE_ADV', None]) for _ in range(num_rows)],
    'Effective_date':[datetime(2023,1,1) + timedelta(days=random.randint(0,365)) for _ in range(num_rows)],
    'Term_Date': [datetime(2024, 1, 1) + timedelta(days=random.randint(0, 730)) if random.random() > 0.7 else None for _ in range(num_rows)],  
    'Client_ID':[random.choice(['OPT_AMZ', 'OPT_WLM', 'OPT_GGL', 'OPT_UHG']) for _ in range(num_rows)],
    'Monthly_Premium': [round(random.uniform(150.0,850.0),2) for _ in range(num_rows)]
}

df = pd.DataFrame(data)
# introduce some messy data  (duplicates and nulls for practice)
df.loc[df.sample(frac = 0.05).index,'Member_id'] = df['Member_id'].iloc[0] #Create duplicates
df.to_csv('optum_eligibility_practice.csv',index = False)
print(filename)

print(f" file '{filename}' created successfully. Size {df.memory_usage(deep = True).sum()/1024**2:.2f} MB")

 optum_eligibility_practice.csv
 file ' optum_eligibility_practice.csv' created successfully. Size 19.59 MB


In [4]:
import sqlite3
#1. load the csv data
df = pd.read_csv('optum_eligibility_practice.csv')
#2. connect to or sqk=lite database 
conn = sqlite3.connect('optum_Practice.db')
cursor = conn.cursor()
#3.Import the dataa name into a table named 'Eligibility'
df.to_sql('Eligibility',conn, if_exists = 'replace',index = False)

#4.create an INDEx (important for performance on 4mb files)
cursor.execute('CREATE INDEX index_member_id on Eligibility (Member_ID)')
print("Database 'Optum_Practice.db' created successfully!")
conn.close()

Database 'Optum_Practice.db' created successfully!


In [5]:
conn = sqlite3.connect('optum_Practice.db')
cursor = conn.cursor()
cursor.execute('''SELECT COUNT(*) AS Total_Rows, 
COUNT(DISTINCT Member_id) as Unique_Member_Count FROM Eligibility;''')
result = cursor.fetchone()
total_rows = result[0]
unique_members = result[1]
print(total_rows)
print("Unique Member:", unique_members)
conn.commit()


50000
Unique Member: 47500


In [6]:
conn = sqlite3.connect('optum_Practice.db')
cursor = conn.cursor()
#cursor.execute('''CREATE TABLE clients(
               #client_id INTEGER PRIMARY KEY AUTOINCREMENT,
               #client_name TEXT NOT NULL,
               #Contract_type  TEXT,
               #Account_Manager TEXT,
               #Region TEXT)''')

cursor.execute('''INSERT INTO Clients(client_name,contract_type,Account_Manager,Region)
               VALUES('Amazon', 'Full-Risk', 'Sarah Jenkins','MN'),
                     ('Walmart', 'ASO', 'Mike Ross','TX'),
                     ('Google', 'Shared-Savings', 'Elena Gilbert','NY')
''');     
conn.commit()
conn.close()      

    

In [7]:
conn = sqlite3.connect('optum_Practice.db')
cursor = conn.cursor()
cursor.execute("""SELECT c.client_name,
                c.Account_Manager, 
               COUNT (e.Member_ID) as Total_enrolled
               FROM eligibility e
               JOIN Clients c ON e.client_id = c.client_id
               GROUP BY c.client_name """)

               

In [8]:
conn = sqlite3.connect('optum_Practice.db')
cursor = conn.cursor()
data = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';",conn)
print(data)

conn.commit()


              name
0          clients
1  sqlite_sequence
2     plan_details
3      claims_data
4      Eligibility


In [9]:
# CLEANING DATA
conn = sqlite3.connect('optum_Practice.db')
cursor = conn.cursor() 
data = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type ='table';" ,conn)

print(df)
leaks = df[(df['Plan_code'].isnull()) & (df['Monthly_Premium'] > 0)]
print(f" 1. revenue leak found: {len(leaks)} cases.")

#2. Check for data logic error 
data_errors = df[df['Term_Date'] <= df['Effective_date']]
print(f" 2. Data equence errors: {len(data_errors)}")

# standarization 
df['Gender'] = df['Gender'].replace({'M': 'Male','F': 'Female','U': 'Unknown'})



      Member_id Subscriber_id First_name Last_name         DOB Gender  \
0       M100000       S104598       Jane   Johnson  1959-10-25      M   
1       M100000       S110866       Jane     Jones  1999-08-09      U   
2       M100002       S104270       Jane     Jones  1997-01-30      M   
3       M100003       S120940       Jane  Williams  1962-04-19      F   
4       M100004       S123619      Sarah     Brown  1978-11-25      F   
...         ...           ...        ...       ...         ...    ...   
49995   M149995       S101036      Sarah   Johnson  2016-11-18      U   
49996   M149996       S118148       Jane     Brown  1995-07-15      U   
49997   M149997       S112432      Sarah     Jones  1993-05-22      M   
49998   M149998       S124246     Emiliy   Johnson  1977-11-06      U   
49999   M149999       S121345     Emiliy  Williams  1980-08-08      F   

          Plan_code Effective_date   Term_Date Client_ID  Monthly_Premium  
0      MEDICARE_ADV     2023-07-28         NaN 

In [10]:
cursor.execute(''' UPDATE eligibility
               SET Plan_code = 'BASIC_HMO'
               WHERE Plan_code IS NULL AND Monthly_Premium > 0
               ''')
cursor.execute(''' UPDATE eligibility
               SET Term_Date = date(Effective_date, '+1 year')
               WHERE Term_Date <= Effective_date ''')
cursor.execute('''UPDATE eligibility 
               SET Gender = 'Unknown' WHERE Gender = 'U'
               ''')
cursor.execute(" SELECT Member_id, COUNT(*) FROM eligibility GROUP BY Member_id")
results = cursor.fetchall()
for Member_id , count in results:
    print(f"{Member_id}: {count}") 

#Having filter groups
cursor.execute('''SELECT Client_id, COUNT(*) as count
               FROM  eligibility 
               GROUP BY Client_id 
               HAVING count > 5
               ''')
result = cursor.fetchall()
for Client_id, count in result:
    print(f"{Client_id}: {count}")

## sum
cursor.execute('SELECT SUM(Monthly_Premium) FROM eligibility')
totalrev = cursor.fetchone()
print(f" total revenue earned {totalrev}")

conn.commit()

M100000: 2501
M100002: 1
M100003: 1
M100004: 1
M100005: 1
M100006: 1
M100008: 1
M100009: 1
M100010: 1
M100011: 1
M100012: 1
M100013: 1
M100014: 1
M100015: 1
M100016: 1
M100017: 1
M100018: 1
M100019: 1
M100020: 1
M100021: 1
M100022: 1
M100023: 1
M100024: 1
M100025: 1
M100026: 1
M100027: 1
M100028: 1
M100029: 1
M100030: 1
M100031: 1
M100032: 1
M100033: 1
M100034: 1
M100037: 1
M100038: 1
M100039: 1
M100040: 1
M100041: 1
M100042: 1
M100043: 1
M100044: 1
M100045: 1
M100047: 1
M100048: 1
M100049: 1
M100050: 1
M100051: 1
M100052: 1
M100053: 1
M100054: 1
M100055: 1
M100056: 1
M100057: 1
M100058: 1
M100059: 1
M100060: 1
M100061: 1
M100062: 1
M100063: 1
M100064: 1
M100065: 1
M100066: 1
M100067: 1
M100068: 1
M100069: 1
M100070: 1
M100071: 1
M100073: 1
M100074: 1
M100075: 1
M100076: 1
M100077: 1
M100078: 1
M100079: 1
M100080: 1
M100081: 1
M100082: 1
M100083: 1
M100084: 1
M100085: 1
M100086: 1
M100087: 1
M100088: 1
M100089: 1
M100090: 1
M100091: 1
M100092: 1
M100093: 1
M100094: 1
M100095: 1
M100096

In [11]:
data = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type ='table';", conn)
print(pd)
print(data)
print(df)
# Verification Check
check_df = pd.read_sql_query("SELECT Gender, COUNT(*) FROM eligibility GROUP BY Gender", conn)
check_termdate = check_df = pd.read_sql_query("SELECT Term_date , COUNT(*) FROM eligibility GROUP BY Term_date", conn)



print(check_df)


<module 'pandas' from 'c:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\pandas\\__init__.py'>
              name
0          clients
1  sqlite_sequence
2     plan_details
3      claims_data
4      Eligibility
      Member_id Subscriber_id First_name Last_name         DOB   Gender  \
0       M100000       S104598       Jane   Johnson  1959-10-25     Male   
1       M100000       S110866       Jane     Jones  1999-08-09  Unknown   
2       M100002       S104270       Jane     Jones  1997-01-30     Male   
3       M100003       S120940       Jane  Williams  1962-04-19   Female   
4       M100004       S123619      Sarah     Brown  1978-11-25   Female   
...         ...           ...        ...       ...         ...      ...   
49995   M149995       S101036      Sarah   Johnson  2016-11-18  Unknown   
49996   M149996       S118148       Jane     Brown  1995-07-15  Unknown   
49997   M149997       S112432      Sarah     Jones  1993-05-22     Male   
49998  

In [12]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('optum_Practice.db')
cursor = conn.cursor()

# Table B: Plan Details (Metadata about the plans)
cursor.execute('''
CREATE TABLE IF NOT EXISTS plan_details (
    Plan_code TEXT PRIMARY KEY,
    Plan_Type TEXT,
    Deductible INTEGER,
    Co_Insurance INTEGER
)''')

# Table C: Claims Data (Transactions/Events)
cursor.execute('''
CREATE TABLE IF NOT EXISTS claims_data (
    Claim_ID TEXT PRIMARY KEY,
    Member_id TEXT,
    Provider_Name TEXT,
    Claim_Amount REAL,
    Diagnosis_Code TEXT,
    FOREIGN KEY (Member_id) REFERENCES eligibility(Member_id)
)''')

# Insert some Master Data
plans = [('PPO_001', 'Premium', 500, 10), ('HMO_CORE', 'Basic', 2000, 20), ('HDHP_SILVER', 'High-Deductible', 4000, 0)]
cursor.executemany("INSERT OR REPLACE INTO plan_details VALUES (?,?,?,?)", plans)

# Insert a few Claims linked to your Members (M100000, etc)
claims = [('C1', 'M100000', 'City Hospital', 1500.0, 'E11.9'), ('C2', 'M100001', 'General Clinic', 250.0, 'J06.9')]
cursor.executemany("INSERT OR REPLACE INTO claims_data VALUES (?,?,?,?,?)", claims)

conn.commit()

### Triple join mastery enrollment, mastery, claims 


In [13]:
query = ('''SELECT
             e.First_name,
             e.Last_name,
             p.Plan_type,
             c.Claim_Amount,
             c.Provider_Name FROM eligibility e
             INNER JOIN plan_details p ON e.Plan_code = p.Plan_code
             INNER JOIN claims_data c ON e.Member_id = c.Member_id; ''')
conn.commit()

In [14]:
#Check the individuals of my data
count_e = pd.read_sql_query(" SELECT COUNT(*) FROM eligibility",conn).iloc[0,0]
count_p = pd.read_sql_query( "SELECT COUNT(*) FROM plan_details", conn).iloc[0,0]
count_c = pd.read_sql_query(" SELECT COUNT(*) FROM claims_data", conn).iloc[0,0]
print(f"Eligibility Rows:{count_e}")
print(f"PLan rows: {count_p}")
print(f" Claims Rows: {count_c}")

#check the joined 
df_joined = pd.read_sql_query(query, conn)
print(f" \n SUccessfully Joined Rows: {len(df_joined)}")

Eligibility Rows:50000
PLan rows: 3
 Claims Rows: 2
 
 SUccessfully Joined Rows: 1488


### Subquerries 


In [22]:
import sqlite3
conn = sqlite3.connect('optum_Practice.db')
cursor = conn.cursor()
 # finding first name 
query = '''SELECT First_name, Last_name, Monthly_Premium
FROM eligibility
WHERE Monthly_Premium > (
SELECT AVG(Monthly_Premium) FROM eligibility);'''
cursor.execute(query)
result = cursor.fetchall()
print(f"\nreflecting data,: {result}")
conn.commit()


reflecting data,: [('Jane', 'Jones', 785.03), ('Jane', 'Williams', 730.07), ('Sarah', 'Brown', 758.66), ('Sarah', 'Jones', 798.41), ('Michael', 'Smith', 755.51), ('Jane', 'Garcia', 710.36), ('John', 'Smith', 624.58), ('John', 'Garcia', 713.72), ('Sarah', 'Smith', 659.16), ('Robert', 'Brown', 729.3), ('John', 'Brown', 650.72), ('Robert', 'Johnson', 778.12), ('Jane', 'Williams', 839.77), ('Michael', 'Brown', 571.16), ('Jane', 'Garcia', 814.18), ('John', 'Garcia', 657.32), ('Robert', 'Brown', 545.52), ('Emiliy', 'Brown', 807.35), ('Michael', 'Johnson', 568.74), ('Jane', 'Jones', 581.08), ('Michael', 'Jones', 849.32), ('Robert', 'Johnson', 736.28), ('Sarah', 'Williams', 641.66), ('Robert', 'Williams', 665.83), ('Jane', 'Garcia', 582.91), ('John', 'Garcia', 512.11), ('Robert', 'Johnson', 564.3), ('Robert', 'Jones', 728.65), ('Sarah', 'Garcia', 662.25), ('Emiliy', 'Williams', 648.69), ('Jane', 'Brown', 759.03), ('Jane', 'Garcia', 628.27), ('John', 'Smith', 837.86), ('Jane', 'Smith', 508.03)

In [25]:
querry = ''' SELECT First_name, Last_name, Monthly_Premium
FROM eligibility
WHERE  Monthly_Premium >( SELECT AVG(Monthly_Premium) FROM eligibility); '''
cursor.execute(query)
result = cursor.fetchall()
print(f"{'First Name':<12} | {'Last Name':<12} | {'Monthly Premium':>15}")
print("-" * 45)
for First_name, Last_name, Monthly_premium in result:
  print(f" {First_name:<12} | {Last_name:<12}| {Monthly_premium: >15.2f}")


First Name   | Last Name    | Monthly Premium
---------------------------------------------
 Jane         | Jones       |          785.03
 Jane         | Williams    |          730.07
 Sarah        | Brown       |          758.66
 Sarah        | Jones       |          798.41
 Michael      | Smith       |          755.51
 Jane         | Garcia      |          710.36
 John         | Smith       |          624.58
 John         | Garcia      |          713.72
 Sarah        | Smith       |          659.16
 Robert       | Brown       |          729.30
 John         | Brown       |          650.72
 Robert       | Johnson     |          778.12
 Jane         | Williams    |          839.77
 Michael      | Brown       |          571.16
 Jane         | Garcia      |          814.18
 John         | Garcia      |          657.32
 Robert       | Brown       |          545.52
 Emiliy       | Brown       |          807.35
 Michael      | Johnson     |          568.74
 Jane         | Jones       |     